In [ ]:
import numpy as np
import nibabel as nib
import os
from nilearn import plotting
from scipy.ndimage import binary_dilation
from os.path import join
from glob import glob

import matplotlib.pyplot as plt

In [ ]:
roi_dir = "./data"
out_dir = "./derivatives"
aub_dir = join(roi_dir, "aub")
hcp_dir = join(roi_dir, "hcp")

In [ ]:
avg_dir = join(out_dir, "avg-masks")
os.makedirs(avg_dir, exist_ok=True)

## Auburn Dataset
Let's load all hand drawn 7T Auburn Habenula masks

In [ ]:
aub_rois = glob(f"{aub_dir}/HIP*.nii.gz", recursive=False)

### Create Auburn masks

In [ ]:
hb_sum = np.zeros_like(nib.load(aub_rois[0]).get_fdata())
for roi in aub_rois:
    roi_arr = nib.load(roi).get_fdata()
    # roi_arr = binary_dilation(roi_arr)
    hb_sum += roi_arr
hb_avg = hb_sum / float(len(aub_rois))

hb_nii = nib.Nifti2Image(hb_sum, nib.load(aub_rois[0]).affine)
hb_avg_nii = nib.Nifti2Image(hb_avg, nib.load(aub_rois[0]).affine)
hb_nii.to_filename(join(avg_dir, "aub_hb_sum.nii.gz"))
hb_avg_nii.to_filename(join(avg_dir, "aub_hb_avg.nii.gz"))

### Create dilated Auburn masks

In [ ]:
hb_sum = np.zeros_like(nib.load(aub_rois[0]).get_fdata())
for roi in aub_rois:
    roi_arr = nib.load(roi).get_fdata()
    roi_arr = binary_dilation(roi_arr)
    hb_sum += roi_arr
hb_avg = hb_sum / float(len(aub_rois))

hb_nii = nib.Nifti2Image(hb_sum, nib.load(aub_rois[0]).affine)
hb_avg_nii = nib.Nifti2Image(hb_avg, nib.load(aub_rois[0]).affine)
hb_nii.to_filename(join(avg_dir, "aub_dil-hb_sum.nii.gz"))
hb_avg_nii.to_filename(join(avg_dir, "aub_dil-hb_avg.nii.gz"))

In [ ]:
template = load_mni152_template(resolution=1)

hcp_rois = glob(f"{hcp_dir}/*kb.nii.gz", recursive=False)

## HCP Dataset 
Let's load all hand drawn 7T Habenula Masks

In [ ]:
hcp_rois = glob(f"{hcp_dir}/*kb.nii.gz", recursive=False)

Resample the HCP masks to be in same space as Auburn

In [ ]:
hcp_rois = hcp_rois = glob(f"{hcp_dir}/*kb.nii.gz", recursive=False)
subjects = [os.path.basename(f).split("-")[0] for f in hcp_rois]
template = join(avg_dir, "aub_hb_avg.nii.gz")
resample_dir = join(roi_dir, "hcp-resampled")

for subject in subjects:
    roi = f"{hcp_dir}/{subject}-hb-kb.nii.gz"
    resamp_hcp = f"{resample_dir}/{subject}_hb-resampled.nii.gz"

    convert = f"3dresample -master {template} -prefix {resamp_hcp} -input {roi}"
    print(convert)
    os.system(convert)

    roi_img = nib.load(resamp_hcp)
    roi_data = roi_img.get_fdata()
    print(f"{roi}: {roi_data.shape}")

Use the resampled masks to create an average and sum HCP mask

In [56]:
hcp_rois = glob(f"{resample_dir}/*resampled.nii.gz", recursive=False)
print(hcp_rois)

hb_sum = np.zeros_like(nib.load(hcp_rois[0]).get_fdata())
for roi in hcp_rois:
    roi_arr = nib.load(roi).get_fdata()
    # roi_arr = binary_dilation(roi_arr)
    hb_sum += roi_arr
hb_avg = hb_sum / float(len(hcp_rois))

hb_nii = nib.Nifti2Image(hb_sum, nib.load(hcp_rois[0]).affine)
hb_avg_nii = nib.Nifti2Image(hb_avg, nib.load(hcp_rois[0]).affine)
hb_nii.to_filename(join(avg_dir, "hcp_hb_sum_resample.nii.gz"))
hb_avg_nii.to_filename(join(avg_dir, "hcp_hb_avg_resample.nii.gz"))

['/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/164636_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/115825_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/581450_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/111514_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/150423_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/826353_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/901442_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/146735_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/156436_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/hb-idconn/data/hcp-resampled/105923_hb-resampled.nii.gz', '/Users/chloehampson/Desktop/projects/h

## RSFC
Katie plotting of the rsfc using masks

In [ ]:
h = plotting.plot_roi(
    hb_avg_nii, draw_cross=False, display_mode="y", cmap="inferno", vmax=0.6
)
h.savefig("hb_mean.png", dpi=600)

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(hb_avg[hb_avg > 0])

In [ ]:
hcp_t = "/Users/katherine.b/Dropbox/Projects/habenula/resting-state/habenula-rsfc-randomise/hcp7t/hcp_96_subj-2019-04-30_sm-0.0mm_tstat1.nii.gz"
auburn_t = "/Users/katherine.b/Dropbox/Projects/habenula/resting-state/habenula-rsfc-randomise/nov20-nilearn_plus_randomise/au_5mm_tstat1.nii.gz"
colin = "/Users/katherine.b/Dropbox/Data/templates/Colin27_T1_seg_MNI-152.nii.gz"

In [ ]:
from nilearn import datasets

fsaverage = datasets.fetch_surf_fsaverage()

In [ ]:
cut_z = [-47, -33, -19, 0, 17, 36, 49]
cut_x = [45, 35, 8, 0, -8, -20, -33, -54]

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(20, 7))

g = plotting.plot_stat_map(
    auburn_t, threshold=4, display_mode="x", cut_coords=cut_x, axes=ax[0]
)
h = plotting.plot_stat_map(
    auburn_t, threshold=4, display_mode="z", cut_coords=cut_z, axes=ax[1]
)
fig.savefig(
    "/Users/katherine.b/Dropbox/Projects/habenula/figures/auburn_randomise_5mm_t>4-slices.png",
    dpi=600,
)

In [ ]:
h = plotting.plot_stat_map(hcp_t, threshold=9)
h.savefig(
    "/Users/katherine.b/Dropbox/Projects/habenula/figures/hcp_randomise_0mm_t>9-ortho.png",
    dpi=600,
)

In [ ]:
plotting.plot_img_on_surf(
    auburn_t,
    threshold=4,
    output_file="/Users/katherine.b/Dropbox/Projects/habenula/figures/auburn_randomise_5mm_t>4-suface.png",
)

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(20, 7))

g = plotting.plot_stat_map(
    hcp_t, threshold=9, display_mode="x", cut_coords=cut_x, axes=ax[0]
)
h = plotting.plot_stat_map(
    hcp_t, threshold=9, display_mode="z", cut_coords=cut_z, axes=ax[1]
)
fig.savefig(
    "/Users/katherine.b/Dropbox/Projects/habenula/figures/hcp_randomise_0mm_t>4-slices.png",
    dpi=600,
)

In [ ]:
plotting.plot_img_on_surf(
    hcp_t,
    threshold=9,
    output_file="/Users/katherine.b/Dropbox/Projects/habenula/figures/hcp_randomise_0mm_t>9-suface.png",
)